In [2]:
import sys
sys.path.append("..")

import os

In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter.ttk import Frame, Label, Button, Checkbutton

class InstallerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Installer")
        self.root.geometry("400x300")
        
        # Initialize variables
        self.current_page = 0
        self.install_dir = tk.StringVar(value="C:/Program Files/MyApp")  # Default installation directory
        self.accept_var = tk.BooleanVar()
        self.component_vars = {
            "Component A": tk.BooleanVar(value=True),
            "Component B": tk.BooleanVar(value=False),
            "Component C": tk.BooleanVar(value=False),
        }
        
        # List of pages
        self.pages = [
            self.create_license_page,
            self.create_component_page,
            self.create_installation_location_page,
        ]
        
        # Main frame for dynamic content
        self.main_frame = Frame(root)
        self.main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Navigation buttons
        self.nav_frame = Frame(root)
        self.nav_frame.pack(fill=tk.X, pady=10)
        
        self.prev_button = Button(self.nav_frame, text="Previous", command=self.previous_page, state=tk.DISABLED)
        self.prev_button.pack(side=tk.LEFT, padx=5)
        
        self.next_button = Button(self.nav_frame, text="Next", command=self.next_page)
        self.next_button.pack(side=tk.RIGHT, padx=5)
        
        # Load the first page
        self.load_page()
    
    def load_page(self):
        """Clears the main frame and loads the current page."""
        for widget in self.main_frame.winfo_children():
            widget.destroy()
        self.pages[self.current_page]()
        
        # Update navigation button states
        self.prev_button.config(state=tk.NORMAL if self.current_page > 0 else tk.DISABLED)
        self.next_button.config(
            text="Install" if self.current_page == len(self.pages) - 1 else "Next"
        )
    
    def next_page(self):
        """Handles navigation to the next page."""
        if self.current_page == 0 and not self.accept_var.get():
            messagebox.showwarning("License Agreement", "You must accept the license agreement to proceed.")
            return
        if self.current_page < len(self.pages) - 1:
            self.current_page += 1
            self.load_page()
        else:
            self.install()
    
    def previous_page(self):
        """Handles navigation to the previous page."""
        if self.current_page > 0:
            self.current_page -= 1
            self.load_page()
    
    def create_license_page(self):
        """Creates the license agreement page."""
        Label(self.main_frame, text="License Agreement", font=("Arial", 14, "bold")).pack(pady=10)
        
        license_text = tk.Text(self.main_frame, height=10, wrap=tk.WORD)
        license_text.insert(tk.END, "Please read the license agreement carefully.\n\n[License terms here...]")
        license_text.config(state=tk.DISABLED)
        license_text.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        Checkbutton(self.main_frame, text="I accept the license agreement", variable=self.accept_var).pack(pady=5)
    
    def create_component_page(self):
        """Creates the component selection page."""
        Label(self.main_frame, text="Select Components to Install", font=("Arial", 14, "bold")).pack(pady=10)
        
        for component, var in self.component_vars.items():
            Checkbutton(self.main_frame, text=component, variable=var).pack(anchor="w", padx=20)
    
    def create_installation_location_page(self):
        """Creates the installation location selection page."""
        Label(self.main_frame, text="Select Installation Location", font=("Arial", 14, "bold")).pack(pady=10)
        
        # Display current installation directory
        entry = tk.Entry(self.main_frame, textvariable=self.install_dir, width=40)
        entry.pack(pady=5, padx=10)
        
        # Button to browse for a directory
        browse_button = Button(self.main_frame, text="Browse", command=self.browse_directory)
        browse_button.pack(pady=5)
    
    def browse_directory(self):
        """Opens a file dialog to select the installation directory."""
        directory = filedialog.askdirectory(initialdir=self.install_dir.get(), title="Select Installation Directory")
        if directory:
            self.install_dir.set(directory)
    
    def install(self):
        """Final installation process."""
        selected_components = [name for name, var in self.component_vars.items() if var.get()]
        messagebox.showinfo(
            "Installation",
            f"Installing components: {', '.join(selected_components)}\n"
            f"Installation directory: {self.install_dir.get()}",
        )
        self.root.quit()

# Run the application
if __name__ == "__main__":
    root = tk.Tk()
    app = InstallerApp(root)
    root.mainloop()
